<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Pr%C3%A9diction_conforme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Projet : Classification et  Prédiction conforme**
Le jeu de données **CIFAR-10** contient 60 000 images en couleur de taille 32x32. Ces images représentent des objets appartenant à 10 classes :

*   avion
*   automobile
*   oiseau
*   chat
*   cerf
*   chien
*   grenouille
*   cheval
*   bateau
*   camion

Le jeu de données d'entraînement contient 50 000 images d'entraînement, soit 5 000 images par classe. Le jeu de test contient 10 000 images de test, soit 1 000 images par classe.

**Pourquoi ce jeu de donnée ?**

CIFAR-10 comme jeu de données est un bon point de départ pour entraîner et tester des Convolutional Neural Networks (CNN), car il n'est pas trop volumineux mais suffisamment varié pour évaluer la performance d'un modèle.

**Importation des données et statistiques simples**

Tu dois dire pourquoi tu choisis ce jeu de donnée. Tu dois également faire de la regression linéaire sur ce jeu de donnée et dire si ça marche ou pas et pourquoi. Vu que tu as une CNN tu peux peut être penser faire de la CV+,  pour faire de la prédiction conforme. Il semble que la SCP serait judicieux

In [ ]:
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# On charge les données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# Division des données en trois ensembles : entraînement, calibration et test
X_train, X_calibration, y_train, y_calibration = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# Normalisation des données
X_train = X_train.astype('float32') / 255.0
X_calibration = X_calibration.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


On entraînne le modèle

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Entraînement du modèle sur l'ensemble d'entraînement
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 50s 86ms/step - accuracy: 0.3312 - loss: 1.8235 - val_accuracy: 0.4980 - val_loss: 1.4141
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 46s 82ms/step - accuracy: 0.5469 - loss: 1.2931 - val_accuracy: 0.5935 - val_loss: 1.1827
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 92s 101ms/step - accuracy: 0.6182 - loss: 1.1028 - val_accuracy: 0.6148 - val_loss: 1.0842
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 71s 82ms/step - accuracy: 0.6476 - loss: 1.0121 - val_accuracy: 0.6275 - val_loss: 1.0474
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 50s 88ms/step - accuracy: 0.6796 - loss: 0.9274 - val_accuracy: 0.6375 - val_loss: 1.0359
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 80s 85ms/step - accuracy: 0.6954 - loss: 0.8880 - val_accuracy: 0.6390 - val_loss: 1.0288
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 83s 88ms/step - accuracy: 0.7086 - loss: 0.8349 - val_accuracy: 0.6718 - val_loss: 0.9620
Epoch 8/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 79s 82ms/step - accuracy: 0.7296 - loss: 0.7774 -

# **La prédiction conforme**

On fait une SCP parceque nous avons beaucoup de données et on cherche à ajuster la distribution des prédictions et fournir un intervalle de confiance pour les prédictions sur les nouvelles données

1. Calcul des scores de non-conformité sur l'ensemble de calibration

In [ ]:
import numpy as np


# Prédictions de calibration
y_pred_calibration_probs = model.predict(X_calibration)

# Extraction des probabilités de la vraie classe
true_class_probs = np.array([y_pred_calibration_probs[i, y_calibration[i]] for i in range(len(y_calibration))])

# Calcul des scores de non-conformité
non_conformity_scores = 1 - true_class_probs


313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step


2. On fixe le niveau de couverture souhaité

In [ ]:
# On fixe le niveau de couverture
alpha = 0.1
quantile_threshold = np.quantile(non_conformity_scores, 1 - alpha)

3. On Applique le seuil de non-conformité sur les nouvelles prédictions

In [ ]:
# Prédictions sur l'ensemble de test
y_pred_test_probs = model.predict(X_test)

# Construction de l'ensemble prédictif pour chaque échantillon de test
prediction_sets = []
for i in range(len(X_test)):
    prediction_set = np.where(y_pred_test_probs[i] >= 1 - quantile_threshold)[0]
    prediction_sets.append(prediction_set)


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


4. On calcul la couverture et la taille moyenne des ensembles prédictifs

In [ ]:
# Calcul de la couverture
correct_cover = [y_test[i] in prediction_sets[i] for i in range(len(y_test))]
coverage = np.mean(correct_cover)

# Calcul de la taille moyenne des ensembles prédictifs
average_set_size = np.mean([len(prediction_set) for prediction_set in prediction_sets])

print(f"Couverture : {coverage * 100:.2f}%")
print(f"Taille moyenne des ensembles prédictifs : {average_set_size:.2f}")


Couverture : 89.21%
Taille moyenne des ensembles prédictifs : 2.13




*   **Couverture de 89,21%**: Cela signifie que, dans environ 9 cas sur 10, la véritable classe est incluse dans l'ensemble prédictif proposé, ce qui est proche de l'objectif de 90%.
*   La **taille moyenne des ensembles prédictifs** indique le nombre moyen de classes incluses dans l'ensemble proposé pour chaque prédiction. Ici, chaque prédiction conforme propose en moyenne 2,13 classes, ce qui signifie que le modèle fournit un ensemble de 2 ou 3 classes en moyenne pour chaque image testée.



# **Regression Linéaire**

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# On charge les données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# On redimensionne les données pour les rendre compatibles avec la régression linéaire
# Convertion des images de 32x32x3 en vecteurs de 32*32*3 = 3072 dimensions
X_full_flat = X_full.reshape((X_full.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

# Division des données en ensembles d'entraînement et de calibration
X_train, X_calibration, y_train, y_calibration = train_test_split(X_full_flat, y_full, test_size=0.2, random_state=42)

# Normalisation des données pour que les pixels soient compris entre 0 et 1
X_train = X_train.astype('float32') / 255.0
X_calibration = X_calibration.astype('float32') / 255.0
X_test_flat = X_test_flat.astype('float32') / 255.0

# On applique une régression linéaire
model = LinearRegression()

# On entraîne le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test_flat)

# Calcul de l'erreur quadratique moyenne (MSE) et le R² pour évaluer les performances
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erreur quadratique moyenne (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")


Erreur quadratique moyenne (MSE) : 8.177422523498535
Coefficient de détermination (R²) : 0.008797228336334229


Une MSE de 8.18 signifie que, en moyenne, les prédictions sont à une distance quadratique de 8.18 des vraies valeurs de classe. Puisque les classes vont de 0 à 9, cela montre que le modèle fait des erreurs assez importantes.

La **régression linéaire** n'est pas un modèle adapté pour ce type de données, car elle suppose une relation linéaire continue entre les pixels et les classes. Or, dans CIFAR-10, les classes sont des labels discrets (catégories d'objets comme des chats, avions, etc.), et il n'existe pas de relation continue ou ordinale entre elles.

# **2ème approche**

In [ ]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Chargement des données CIFAR-10
(X_full, y_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Prétraitement des données
# Normalisation des pixels dans l'intervalle [0, 1]
X_full = X_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Encodage one-hot des labels
y_full = to_categorical(y_full, 10)
y_test = to_categorical(y_test, 10)

# Division des données d'entraînement pour avoir un ensemble de validation
X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Construction du modèle CNN
model = Sequential([
    # Première couche convolutionnelle
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),

    # Deuxième couche convolutionnelle
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Troisième couche convolutionnelle
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Aplatissement
    Flatten(),

    # Couche entièrement connectée
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compilation du modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Affichage de la structure du modèle
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,202 (625.79 KB)

 Trainable params: 160,202 (625.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entraînement du modèle
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, y_val)
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 100ms/step - accuracy: 0.2656 - loss: 1.9439 - val_accuracy: 0.4891 - val_loss: 1.4028
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 106ms/step - accuracy: 0.4867 - loss: 1.4206 - val_accuracy: 0.5629 - val_loss: 1.2177
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 95ms/step - accuracy: 0.5596 - loss: 1.2457 - val_accuracy: 0.6111 - val_loss: 1.0807
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 85s 101ms/step - accuracy: 0.6073 - loss: 1.1313 - val_accuracy: 0.6363 - val_loss: 1.0139
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 97ms/step - accuracy: 0.6400 - loss: 1.0349 - val_accuracy: 0.6527 - val_loss: 0.9947
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 98ms/step - accuracy: 0.6624 - loss: 0.9649 - val_accuracy: 0.6609 - val_loss: 0.9569
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 95ms/step - accuracy: 0.6902 - loss: 0.8901 - val_accuracy: 0.6727 - val_loss: 0.9220
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 97ms/step - accuracy: 0.7100 - loss: 0.8385

In [ ]:
# Évaluation sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
test_loss, test_accuracy

313/313 - 4s - 12ms/step - accuracy: 0.7029 - loss: 0.8810


(0.8809875845909119, 0.7028999924659729)

# **PREDICTION CONFORME**

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10

# Chargement des données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# Prétraitement
X_full = X_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_full = y_full.flatten()

# Division des données : Entraînement, Calibration, Test
X_train, X_calib, y_train, y_calib = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [2]:
# Modèle CNN (reprendre le modèle précédent)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Encodage des labels pour l'entraînement
y_train_onehot = to_categorical(y_train, 10)
y_calib_onehot = to_categorical(y_calib, 10)
y_test_onehot = to_categorical(y_test, 10)

# Construction du modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Compilation et entraînement
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_onehot, epochs=10, batch_size=64, validation_data=(X_calib, y_calib_onehot))

# Prédictions sur l'ensemble de calibration
calib_probs = model.predict(X_calib)

# Calcul des scores de non-conformité pour l'ensemble de calibration
calib_scores = 1 - np.max(calib_probs, axis=1)

# Déterminer le seuil pour le quantile (90% de couverture théorique)
quantile = 0.9
threshold = np.quantile(calib_scores, quantile)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 99ms/step - accuracy: 0.2657 - loss: 1.9607 - val_accuracy: 0.4872 - val_loss: 1.4301
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 101ms/step - accuracy: 0.4937 - loss: 1.4046 - val_accuracy: 0.5658 - val_loss: 1.2070
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 98ms/step - accuracy: 0.5572 - loss: 1.2483 - val_accuracy: 0.6031 - val_loss: 1.1074
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 98ms/step - accuracy: 0.6040 - loss: 1.1307 - val_accuracy: 0.6371 - val_loss: 1.0177
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 96ms/step - accuracy: 0.6395 - loss: 1.0251 - val_accuracy: 0.6544 - val_loss: 0.9882
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.6710 - loss: 0.9460 - val_accuracy: 0.6783 - val_loss: 0.9181
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 96ms/step - accuracy: 0.6917 - loss: 0.8915 - val_accuracy: 0.6815 - val_loss: 0.9105
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 97ms/step - accuracy: 0.7073 - loss: 0.8342 -

In [4]:
# Prédictions conformes sur l'ensemble de test
test_probs = model.predict(X_test)
test_scores = 1 - np.max(test_probs, axis=1)

# Définir des intervalles prédictifs conformes
test_intervals = [
    np.where(test_probs[i] >= 1 - threshold)[0] for i in range(len(test_probs))
]

# Évaluer la couverture empirique
true_labels_in_intervals = [
    y_test[i] in test_intervals[i] for i in range(len(y_test))
]
empirical_coverage = np.mean(true_labels_in_intervals)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step


In [5]:
print(f"Theoretical Coverage: {quantile}")
print(f"Empirical Coverage: {empirical_coverage}")

Theoretical Coverage: 0.9
Empirical Coverage: 0.6981


# **Visualisations**

In [6]:
# Prédictions sur un sous-ensemble de test pour examiner les cas individuels
import pandas as pd

# Sélection d'un petit sous-ensemble
subset_indices = np.random.choice(len(X_test), size=5, replace=False)
subset_X = X_test[subset_indices]
subset_y = y_test[subset_indices]

In [7]:
# Prédictions et scores pour ces indices
subset_probs = model.predict(subset_X)
subset_scores = 1 - np.max(subset_probs, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [8]:
# Définir les intervalles prédictifs pour ces cas
subset_intervals = [
    np.where(subset_probs[i] >= 1 - threshold)[0] for i in range(len(subset_probs))
]

In [9]:
# Rassembler les résultats pour visualisation
results = []
for i, index in enumerate(subset_indices):
    results.append({
        "Index": index,
        "True Label": subset_y[i][0],
        "Predicted Probabilities": subset_probs[i],
        "Prediction Interval": subset_intervals[i]
    })

# Conversion en DataFrame pour un affichage clair
results_df = pd.DataFrame(results)

In [11]:
results_df

,Index,True Label,Predicted Probabilities,Prediction Interval
0,8306,6,"[0.009561936, 0.0002111212, 0.78589565, 0.0354...",[2]
1,9386,4,"[0.0005299588, 2.8325732e-07, 0.8771177, 0.005...",[2]
2,169,0,"[0.56842434, 0.00047006027, 0.10543867, 0.0025...",[0]
3,2182,4,"[0.0002819759, 1.202852e-05, 0.040843174, 0.08...",[4]
4,4284,9,"[6.391249e-09, 0.00081446447, 3.5197307e-11, 3...",[9]
